### Batting Analysis

Uncomment this line to get the most recent statistics from the DakStats website.

In [18]:
#%run ./Conference_Statistics.ipynb

As always, import the necessary statements. Additionally, we change our pandas settings to show all the columns for all of our dataframes, allowing us to view all of the data.

In [2]:
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)

We import our dataframes for batting, pitching, and fielding statistics as well as our list of teams from the pickle file titled `Stats.pkl`.

In [3]:
with open('Stats.pkl', 'rb') as f:
    dfb = pickle.load(f)
    dfp = pickle.load(f)
    dff = pickle.load(f)
    teams = pickle.load(f)

Here, we create a function to calculate some common baseball statistics as well as a few of the more advanced metrics we will be using in our analysis. This function will be useful later on when we gather our league totals so that we do not have to write up all of these same calculations for the new dataframe.

In [4]:
def calcBatStats(df):
    # Batting Average (useful later for League totals)
    df["AVG"] = round(df["H"] / df["AB"], 3)
    # Caught Stealing
    df["CS"] = round(df["SBA"] - df["SB"], 3)
    # Plate Appearances    
    df["PA"] = round(df["AB"] + df["SF"] + df["SH"] + df["BB"] + df["HBP"])
    # OPS
    df["OPS"] = round(df["OBP"] + df["SLG"], 3)
    # Singles
    df["1B"] = df["H"] - df["2B"] - df["3B"] - df["HR"]
    # Runs Created
    df["RC"] = round((df["H"] + df["BB"]) * df["TB"] / (df["AB"] + df["BB"]), 3)
    # POP
    df["POP"] = round(df["OPS"] + df["AVG"], 3)
    # Total Average
    df["TA"] = round((df["TB"] + df["BB"] + df["HBP"] + df["SB"]) / (df["AB"] - df["H"] + df["SH"] + df["SF"] + df["CS"] + df["GDP"]), 3)
    # Batting Average on Balls in Play
    df["BABIP"] = round((df["H"] - df["HR"]) / (df["AB"] - df["SO"] - df["HR"] + df["SF"]), 3)
    # Hoban Efficiency Quotient - Offense
    df["HEQO"] = df["TB"] + df["R"] + df["RBI"] + df["SB"] + (0.5 * df["BB"])
    # Base Stealing Runs
    df["BSR"] = round(((df["H"] + df["BB"] - df["CS"]) * (df["TB"] + (0.7 * df["SB"]))) / (df["AB"] + df["BB"] + df["CS"]), 3)

We run this function on our batting dataframes for each team. Note the newly created columns on the right side of the table. 

In [5]:
for df in dfb:
    calcBatStats(df)
dfb[2][:5]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
1,"Danforth, Patrick",32,28.0,0.354,99,18,35,6,3,2,13,53,0.535,9,6,34,2,0.435,1,3,12,12,0,118,0.970,24,21.593,1.324,1.143,0.516,100.5,25.015
2,"Griffin, Chris",32,28.0,0.339,109,19,37,8,0,6,18,63,0.578,17,1,26,0,0.426,2,1,2,2,0,130,1.004,23,27.000,1.343,1.107,0.392,110.5,27.600
3,"Carr, Austin",32,28.0,0.284,109,12,31,4,1,2,20,43,0.394,10,4,38,1,0.366,0,1,0,0,0,124,0.760,24,14.815,1.044,0.712,0.420,80.0,14.815
4,"Newkirk, Sam",32,27.0,0.255,106,6,27,12,1,2,13,47,0.443,4,5,21,1,0.313,0,0,1,1,0,115,0.756,12,13.245,1.011,0.712,0.301,69.0,13.443
5,"Pawlik, Jeff",32,28.0,0.225,102,13,23,6,0,0,12,29,0.284,7,6,30,1,0.310,1,0,1,2,1,116,0.594,17,7.982,0.819,0.524,0.315,58.5,7.830


#### Crossroads League Statistics

Now that we have a few more advanced individual batting statistics, we need to gather the Crossroads League totals in order to incorporate them into our statistics and eventually rank the various hitters based on different stats.

To do this, we first subset each of the teams' batting table to extract only the Total row at the bottom. 

In [6]:
bat_totals = [df[df.Batting.str.contains("Total:", regex = False)] for df in dfb]
bat_totals[2]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
31,Total:,32,NaN,0.233,959,142,223,52,6,14,122,329,0.343,105,38,294,10,0.33,8,12,37,53,16,1122,0.673,151,101.421,0.906,0.651,0.317,682.5,102.527


Next, we combine each of these rows into a new dataframe named `merged_bat_totals` using the code `pd.concat`. Below we show the first four rows of this new table.

In [7]:
merged_bat_totals = pd.concat(bat_totals)
merged_bat_totals[:4]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
26,Total:,36,NaN,0.240,1059,142,254,47,5,17,124,362,0.342,133,31,274,22,0.339,9,19,34,42,8,1251,0.681,185,117.529,0.921,0.649,0.305,728.5,121.849
32,Total:,28,NaN,0.208,814,71,169,22,2,3,67,204,0.251,91,19,275,7,0.299,8,6,19,25,6,938,0.550,142,58.608,0.758,0.496,0.305,406.5,60.586
31,Total:,32,NaN,0.233,959,142,223,52,6,14,122,329,0.343,105,38,294,10,0.330,8,12,37,53,16,1122,0.673,151,101.421,0.906,0.651,0.317,682.5,102.527
28,Total:,36,NaN,0.306,1125,256,344,78,10,33,226,541,0.481,124,50,215,11,0.393,20,17,92,108,16,1336,0.874,223,202.713,1.180,0.955,0.347,1177.0,216.317


Now that we have our Totals table, we add a total row at the bottom using the code `.sum()`. Since the column labeled "Batting" does not provide any valuable information, we remove it from the table. Then, we isolate the totals row at the bottom with the code `.iloc[-1:]` to put the batting totals into a series.

In [8]:
merged_bat_totals.loc["CL_Total"] = merged_bat_totals.sum()
del merged_bat_totals["Batting"]
CL_bat_totals = merged_bat_totals.iloc[-1,:]

We make a copy to avoid any errors from overwriting the values, and then we calculate a few more statistics and run the `calcBatStats` function on the series. We printed out a few of the league totals.

In [9]:
CL_tot_b = CL_bat_totals.copy()
# CL On-Base Percentage
CL_tot_b["OBP"] = round((CL_tot_b["H"] + CL_tot_b["BB"] + CL_tot_b["HBP"]) / (CL_tot_b["AB"] + CL_tot_b["BB"] + CL_tot_b["HBP"] + CL_tot_b["SF"]), 3)
# CL Slugging Percentage
CL_tot_b["SLG"] = round(CL_tot_b["TB"] / CL_tot_b["AB"], 3)
# CL weighted On-Base Average (excluding IBB, formula from https://library.fangraphs.com/offense/woba/)
CL_tot_b["wOBA"] = round((0.69*CL_tot_b["BB"] + 0.72*CL_tot_b["HBP"] + 0.89*CL_tot_b["1B"] + 1.27*CL_tot_b["2B"] + 1.62*CL_tot_b["3B"] + 2.10*CL_tot_b["HR"]) / (CL_tot_b["AB"] + CL_tot_b["BB"] + CL_tot_b["SF"] + CL_tot_b["HBP"]), 3)
# CL Runs per Plate Appearance
CL_tot_b["R_per_PA"] = round(CL_tot_b["R"] / CL_tot_b["PA"], 3)
CL_tot_b["wRC_per_PA"] = CL_tot_b["R_per_PA"]
# ABF (statistic that incorporates the value of various methods of reaching base with regard to their assistance towards scoring runs)
CL_tot_b["ABF"] = round((0.47*CL_tot_b["1B"] + 0.38*CL_tot_b["2B"] + 0.55*CL_tot_b["3B"] + 0.93*CL_tot_b["HR"] + 0.33*CL_tot_b["BB"] + CL_tot_b["HBP"]) / (CL_tot_b["AB"] - CL_tot_b["H"]), 3)
# Calculate other totals such as AVG, OPS, POP, etc
calcBatStats(CL_tot_b)
# deleted Games Started column because it doesn't apply to league totals
del CL_tot_b["GS"]

In [10]:
CL_tot_b[:5]

GP       336.000
AVG        0.274
AB     10288.000
R       1950.000
H       2815.000
Name: CL_Total, dtype: float64

#### Advanced Statistics/Ranking Hitters

We would like to be able to rank hitters across the Crossroads League, and we will do this with a statistic called Weighted Runs Created Plus (wRC+). Below we calculate a few other statistics that will get us closer to our calculation of wRC+ for each hitter. 

In [11]:
for df in dfb:
    #OPS+
    df["OPS_plus"] = round(100 * ((df["OBP"]/CL_tot_b["OBP"]) + (df["SLG"]/CL_tot_b["SLG"]) - 1), 3)
    #Weighted On-Base Average
    df["wOBA"] = round(((0.69*df["BB"]) + (0.72*df["HBP"]) + (0.89*df["1B"]) + (1.27*df["2B"]) + (1.62*df["3B"]) + (2.1*df["HR"])) / (df["AB"] + df["BB"] + df["SF"] + df["HBP"]), 3)
    #Batting Runs
    df["Bat_Runs"] = round(((0.69*df["BB"]) + (0.72*df["HBP"]) + (0.89*df["1B"]) + (1.27*df["2B"]) + (1.62*df["3B"]) + (2.1*df["HR"])) / (df["AB"] + df["BB"] + df["SF"] + df["HBP"]), 3)
    #Weighted Runs Above Average (verify constant)
    df["wRAA"] = round(((df["wOBA"] - CL_tot_b["wOBA"]) / (1.157)) * df["PA"], 3)
    #Weighted Runs Created
    df["wRC"] = round(df["wRAA"] + (df["PA"] * (CL_tot_b["R"])/(CL_tot_b["PA"])), 3)
    #K/9
    df["K_pct"] = round(df["SO"]/df["AB"],2)

Since wRC+ incorporates the park factor for each hitter's home park, we have to load our various park factors for each team.

In [12]:
with open('Park_Factor.pkl', 'rb') as f:
    park_factor = pickle.load(f)

Now that we have our park factors, we can calculate our wRC+ for each hitter.

In [13]:
#Weighted Runs Created+
for i in range(len(teams)):
    dfb[i]["wRC_plus"] = round(100 * ((((dfb[i]["wRAA"] / dfb[i]["PA"]) + CL_tot_b["R_per_PA"]) + (CL_tot_b["R_per_PA"] - (park_factor[i]) * (CL_tot_b["R_per_PA"]))) / (CL_tot_b["wRC_per_PA"])), 3)

Before we begin ranking the hitters, we subset the data to only include hitters that have at least 27 at bats, which equates to one per conference game. 

In [14]:
#make sure everyone has at least 16 at bats (1 per game)
for i in range(len(teams)):
    dfb[i] = dfb[i][dfb[i]['AB'] >= 32]

We make copies of our dataframes to avoid errors and then we pretty the dataframes by adding a column for each player's team, removing the totals and opponents rows, rearranging the table's columns to show the important information first, and removing the unneccesary decimal in the `GS` column.

In [15]:
temp_dfb = [] 
for df in dfb: #make copy to avoid errors
    temp_dfb.append(df.copy())
for i in range(len(teams)): #add column for team
    temp_dfb[i]["Team"] = teams[i]
for df in temp_dfb: 
    df.drop(df.tail(2).index,inplace=True) # drop last 2 rows (only run this line once or data will be lost)
    team = df.pop("Team")
    df.insert(1, team.name, team) #move team column to second
    k = df.pop("K_pct")
    df.insert(16, k.name, k) #move team column to second
    stat = df.pop("wRC_plus")
    df.insert(4, stat.name, stat) #move wRC+ column to front of statistical columns
    df['GS'] = df['GS'].astype(int) #remove decimal place on GS column
temp_dfb[2][-2:] #verify totals and opponents rows are gone

,Batting,Team,GP,GS,wRC_plus,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,K_pct,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR,OPS_plus,wOBA,Bat_Runs,wRAA,wRC
12,"Mullet, Jaron",Grace,30,20,56.734,0.109,64,14,7,2,0,0,7,9,0.141,19,1,0.55,35,0,0.318,1,1,3,4,1,86,0.459,5,2.819,0.568,0.533,0.233,42.5,3.304,21.123,0.245,0.245,-7.507,6.290
14,"Buschman, Grant",Grace,16,11,19.456,0.079,38,2,3,1,0,0,4,4,0.105,5,2,0.53,20,2,0.222,0,1,0,0,0,46,0.327,2,0.744,0.406,0.289,0.167,12.5,0.744,-13.837,0.176,0.176,-6.759,0.621


With our more readable data, we combine each team's table into one dataframe wiht the code `pd.concat`. Then, we sort the table by wRC+ to show the hitters with the highest wRC+ first. Finally, we display our top ten hitters.

In [16]:
all_hitters = pd.concat(temp_dfb) #collect all in one dataframe
top_hitters = all_hitters.sort_values(by=['wRC_plus'], ascending=False) #sort by wRC+ in descending order
top_hitters[:10] #top 10 hitters

,Batting,Team,GP,GS,wRC_plus,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,K_pct,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR,OPS_plus,wOBA,Bat_Runs,wRAA,wRC
3,"Dice, Camden",HU,36,36,185.138,0.374,115,23,43,15,0,4,31,70,0.609,15,3,0.25,29,0,0.452,2,1,2,2,0,136,1.061,24,31.231,1.435,1.200,0.464,133.5,31.855,172.061,0.454,0.454,12.695,34.513
4,"Lichty, Daniel",HU,36,36,178.115,0.348,132,33,46,9,1,8,33,81,0.614,15,5,0.08,10,2,0.423,4,0,24,26,2,156,1.037,28,33.612,1.385,1.330,0.322,178.5,38.726,165.381,0.441,0.441,12.809,37.836
2,"Wilson, Satchell",HU,36,36,175.954,0.378,127,31,48,8,2,3,28,69,0.543,13,8,0.11,14,3,0.460,2,2,18,21,3,152,1.003,35,30.064,1.381,1.213,0.402,152.5,33.097,158.103,0.437,0.437,11.955,36.340
3,"Saal, Aaron",MVNU,32,32,174.247,0.360,111,27,40,8,1,8,33,74,0.667,14,1,0.11,12,2,0.423,4,0,12,14,2,130,1.090,23,31.968,1.450,1.278,0.337,153.0,33.739,178.340,0.457,0.457,12.472,33.328
1,"Porcellato, Matteo",Marian,36,36,173.009,0.373,126,31,47,11,1,1,28,63,0.500,20,12,0.17,22,1,0.497,1,1,11,13,2,160,0.997,34,28.911,1.370,1.262,0.442,143.0,31.051,157.672,0.443,0.443,13.414,39.082
5,"Glover, Jake",MVNU,32,32,167.765,0.342,111,17,38,9,0,7,38,68,0.613,15,5,0.10,11,2,0.433,3,0,0,2,2,134,1.046,22,28.603,1.388,1.100,0.323,130.5,27.094,167.861,0.445,0.445,11.466,32.963
5,"Thixton, Tye",IWU,32,32,167.188,0.373,134,45,50,10,1,10,35,92,0.687,10,3,0.10,14,3,0.429,0,1,11,11,0,148,1.116,29,38.333,1.489,1.318,0.364,188.0,41.542,184.864,0.477,0.477,16.757,40.500
6,"Blinn, Denver",IWU,32,32,166.106,0.366,123,45,45,16,3,5,22,82,0.667,12,8,0.15,18,0,0.451,1,0,13,15,2,144,1.118,21,34.622,1.484,1.420,0.396,168.0,36.573,185.969,0.475,0.475,16.055,39.157
4,"Goodin, Lucas",IWU,32,32,159.085,0.374,107,29,40,7,1,4,30,61,0.570,21,3,0.13,14,0,0.489,0,0,6,9,3,131,1.059,28,29.070,1.433,1.300,0.404,136.5,28.867,172.607,0.462,0.462,13.134,34.150
7,"Killian, Tanner",IWU,30,30,158.545,0.352,105,24,37,11,0,8,38,72,0.686,5,9,0.26,27,2,0.415,4,1,1,1,0,124,1.101,18,27.491,1.453,1.160,0.392,137.5,27.758,180.805,0.461,0.461,12.325,32.218
